<h1 style='font-size:40px'> NPL Risk Evaluation Modeling</h1>
<div style='font-size:20px'> 
    <ul> 
        <li> 
            This project aims the conceiving of a Machine Learning Model focused on assisting a bank on its credit approval strategy.
        </li>
        <li> 
            The corporation has been scolded for its recent NPL levels by its shareholders. Thus, the executive team has decided that a more conservative 
            credit strategy must be adopted for new contracts.
        </li>
        <li> 
            During the planning meetings, the business team has made two major requests concerning the nature of the model.
            <ul style='list-style-type:decimal'> 
                <li> 
                    It must be focused on predicting whether a given client might produce an NPL in the future.
                </li>
                <li> 
                    The output must be some kind of score suggesting the likelihood of the event to happen. They are not looking for 
                    an incisive "yes or no" answer.
                </li>
            </ul>
        </li>
    </ul>
    <p style='margin-left:30px'> <strong> Note:</strong> The bank's NPL definition is any loan which payment is at least 90 days late.</p>
</div>

<h2 style='font-size:30px'> Data Importing</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The Data Engineers were able to provide two .csv views from the bank's database. The first one contains general information over the clients 
            and the second lists the loans they've contracted over some period of time.
        </li>
    </ul>
</div>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=6f0fc2e6c3cc65c3aed68eeaf7ef74ec20ffc9b57e99f4dcbac111f2836834f2
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from IPython.core.display import HTML

# Creating the project's SparkSession.
spark = SparkSession.builder.appName('NPL').getOrCreate()

# Also, modifying the session's log level.
log_level = spark.sparkContext.setLogLevel('ERROR')

# This tiny config enables us to scroll along the DataFrame's columns.
display(HTML("<style>pre { white-space: pre !important; }</style>"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/15 22:50:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3 style='font-size:30px;font-style:italic'> Clients Database</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            This dataset is comprised of general information about the loans' clients.
        </li>    
        <li> 
            A particularity worth noting is that date columns show the negative amount of days since the given event took place. Positive numbers 
            indicate the number of days since the occurence ceased to exist - as it might happen with unemployed borrowers in the DAYS_EMPLOYED feature.
        </li>
    </ul>
</div>

In [3]:
path_clients = '/kaggle/input/credit-card-approval-prediction/application_record.csv'

# Defining the data types from the clients dataset.
schema_clients = '''
`ID` STRING, `CODE_GENDER` STRING, `FLAG_OWN_CAR` STRING, `FLAG_OWN_REALTY` STRING, `CNT_CHILDREN` INT,
`AMT_INCOME_TOTAL` FLOAT, `NAME_INCOME_TYPE` STRING, `NAME_EDUCATION_TYPE` STRING, `NAME_FAMILY_STATUS` STRING, `NAME_HOUSING_TYPE` STRING,
`DAYS_BIRTH` INT, `DAYS_EMPLOYED` INT, `FLAG_MOBIL` STRING, `FLAG_WORK_PHONE` STRING, `FLAG_PHONE` STRING, `FLAG_EMAIL` STRING, 
`OCCUPATION_TYPE` STRING, `CNT_FAM_MEMBERS` DOUBLE
'''

# Reading the database with the created schema.
df_clients = spark.read.csv(path_clients, header=True, schema=schema_clients)
df_clients.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Duplicates Disclaimer</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
             Clients may not have unique rows in the dataset because the ID column identifies a contracted loan instead of a person.
        </li>
        <li> 
            Thus, I've found convenient for the project to create an ID column that assigns a code for each of the clients.
        </li>
    </ul>
</div>

In [4]:
# Another issue unnoticed by the Data Engineers is that the database contains repeated Loan ID's.
from pyspark.sql.functions import max as ps_max

# Observe that there are Loans mentioned two times. It would be proper to disconsider them. 
data_duplicate_id = (df_clients
                     .groupBy('ID')
                     .count()
                     .filter('`count`>1')
                            )
data_duplicate_id.show(5)

+-------+-----+
|     ID|count|
+-------+-----+
|7742298|    2|
|7174719|    2|
|7091721|    2|
|7089090|    2|
|7022197|    2|
+-------+-----+
only showing top 5 rows



In [5]:
# If we take a peek on the first mentioned ID, we can notice that the presented loan actually is assigned to two different people!

# Thus, the presence of such deals is potentially harmful for our model. It would be sensible to discard such ID's from the database.
df_clients.filter('`ID`==7742298').show()

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|7742298|          F|           N|              Y|           0|        144000.0|         Working|Secondary / secon...|             Widow|House / apartment|    -20626|        -1455|         1|              0|       

In [6]:
# Dropping out the problematic loans.
df_clients = data_duplicate_id.join(df_clients, how='right', on='ID').where('`count` IS NULL').drop('count')

In [7]:
# Listing the `df_clients` features with the exception of ID.
features_clients = df_clients.columns
features_clients.remove('ID')
features_clients

['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS']

In [8]:
# Now, getting back to the Client's ID issue, I'd like to present a brief analysis on it.
# Note that the database's actual amount of clients is lower than its number of rows. 
data_clients = df_clients.dropDuplicates(features_clients) 
print(f'`df_clients` length: {df_clients.count()}')
print(f'Number of clients: {data_clients.count()}')

`df_clients` length: 438463


Number of clients: 90084


In [9]:
# We'll assign an ID for every client mentioned in `df_clients`. 
from pyspark.sql.functions import cast, row_number
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

window = Window.orderBy(features_clients)
row_window = row_number().over(window)

# A DataFrame with the clients' data and actual ID.
df_id_clients = data_clients.withColumn('ID_CLIENT', row_window.cast(StringType())).drop('ID')

In [10]:
# We'll need to perform Null Safe JOIN's, since columns such as 'OCCUPATION_TYPE' contain null values.
from functools import reduce

# Creating the multiple null safe JOIN's condition.
condition_id_client = reduce(lambda x,y: x&y, [df_clients[col].eqNullSafe(df_id_clients[col]) for col in features_clients])
columns_join_id = ['df_clients.*', 'df_id_clients.ID_CLIENT'] # Listing the JOIN columns.

# Consolidating the final clients database.
df_clients = (df_clients.alias('df_clients') # Resorting to aliases for both DataFrames present columns with same names.
                 .join(df_id_clients.alias('df_id_clients'), condition_id_client)
                 .select(columns_join_id))
df_clients.show(5)

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|ID_CLIENT|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---------+
|5996382|          F|           N|              N|           0|         28800.0|       Pensioner|Secondary / secon...|           Married|House / apartment|    -20298|       365243|    

<h3 style='font-size:30px;font-style:italic'> Loans Database</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            This table contains the payments records for every loan since its contraction. 
        </li>
        <li> 
            But in order to the dataset be adequate to our project's intent, two transformations are necessary: first, we need to bring the `ID_CLIENT`
            column to it and after that, group the database so that it denounces individuals who've produced an NPL at least once.            
        </li>
    </ul>
</div>

In [11]:
# Bringing the dataset into our notebook.
path_loans = '/kaggle/input/credit-card-approval-prediction/credit_record.csv'
schema_loans = '`ID` STRING, `MONTHS_BALANCE` INT, `STATUS` STRING'
df_loans = spark.read.csv(path_loans, header=True, schema=schema_loans)
df_loans.show(5) 

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
+-------+--------------+------+
only showing top 5 rows



In [12]:
# Now, providing the loans' client ID.
df_loans = df_loans.join(df_clients, ['ID']).select(['ID_CLIENT', 'ID', 'MONTHS_BALANCE', 'STATUS'])
df_loans.show(5)

+---------+-------+--------------+------+
|ID_CLIENT|     ID|MONTHS_BALANCE|STATUS|
+---------+-------+--------------+------+
|    34269|5008810|             0|     C|
|    34269|5008810|            -1|     C|
|    34269|5008810|            -2|     C|
|    34269|5008810|            -3|     C|
|    34269|5008810|            -4|     C|
+---------+-------+--------------+------+
only showing top 5 rows



<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Conceiving the Target Variable</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The `STATUS` column presents a handful of codes that represent distinct status for a loan's payment. Their definition is as follows:
            <table style='font-size:15px;margin-top:20px'> 
                <tr>
                    <th> Code</th>
                    <th> Definition</th>
                </tr>
                <tr> 
                    <td> C</td>
                    <td> Paid off that month</td>
                </tr>
                <tr> 
                    <td> 0</td>
                    <td> 1-29 days past due</td>
                </tr>
                <tr> 
                    <td> 1</td>
                    <td> 30-59 days past due </td>
                </tr>
                <tr> 
                    <td> 2</td>
                    <td> 60-89 days past due </td>
                </tr>
                <tr> 
                    <td> 3</td>
                    <td> 90-119 days past due </td>
                </tr>
                <tr> 
                    <td> 4</td>
                    <td> 120-149 days past due </td>
                </tr>
                <tr> 
                    <td> 5</td>
                    <td> Overdue or bad debts,<p> write-offs for more than 150 days</p> </td>
                </tr>
                <tr> 
                    <td> X</td>
                    <td> No loan for the month</td>
                </tr>
            </table>
        </li>
        <li style='margin-top:20px'> 
            Observe that in our case only the 3, 4 and 5 codes are of our interest. Thus it would be convenient to create a binary flag that denounces whether 
            the individual has ever caused an NPL.
        </li>
    </ul>
</div>

In [13]:
# The dependent variable's conception needs a custom GroupBy that PySpark is unable to perform by itself. Hence, we 
# are going to resort to pandas in this section.
import pandas as pd

# Defining the GroupBy's schema.
schema_flag_npl = '`ID_CLIENT` STRING, `NPL` STRING'

# This lambda expression signs whether a client has ever produced an NPL in the past.
lambda_npl = lambda x: '1' if x.STATUS.isin(['3', '4', '5']).any() else '0'

def has_npl(df:pd.DataFrame)->pd.DataFrame:
    '''
        Verifies if a client's  records contain any sort of Non-Performing Loan.
        
        Parameter
        ---------
        `df`: pd.DataFrame 
            The loan records of a certain client.
        
        Returns
        -------
        A `pd.DataFrame` with the client's ID and a flag indicating NPL existence in their loan history. 
    '''
    output = df.groupby(['ID_CLIENT']).apply(lambda_npl) # `lambda_npl` takes care of the flags creation.
    output.name = 'NPL' # Setting the flag column's name.
    return output.reset_index()

# Finally, generating our target-variable.
target = df_loans.groupBy('ID_CLIENT').applyInPandas(has_npl, schema_flag_npl)

<h2 style='font-size:30px'> Consolidating the Data</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            With both datasets properly treated, we are able to JOIN them in a single table.
        </li>
    </ul>
</div>

In [14]:
# Finally, enriching the clients information with the NPL flag.
df = df_id_clients.join(target, how='inner', on='ID_CLIENT')

In [15]:
#df.write.parquet('/kaggle/input/df-parquet', mode='overwrite')

<h2 style='font-size:30px'> Dataset Splitting</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            With the dataset properly treated, we are able to begin our EDA and model creation. But firstly we have to separate the data in 
            the training and test tables.
        </li>
    </ul>
</div>

In [16]:
# As we can see, we are dealing with an unbalanced dataset case. Thus it is interesting to maintain the target value proportions
# in both training and test sets.
from pyspark.sql.functions import col

df.groupBy('NPL').count().withColumn('proportion', col('count')/df.count()).show()

+---+-----+-------------------+
|NPL|count|         proportion|
+---+-----+-------------------+
|  0| 9509| 0.9774876644736842|
|  1|  219|0.02251233552631579|
+---+-----+-------------------+



In [17]:
# Creating the training and test sets.
train = df.sampleBy('NPL', fractions={'0':.75, '1':.75}, seed=42)
test = df.subtract(train)

In [18]:
# Since the class proportions are relatively proximate, we are able to use them in our project.
print('*** TRAIN ***')
train.groupBy('NPL').count().withColumn('proportion', col('count')/train.count()).show()
print('*** TEST ***')
test.groupBy('NPL').count().withColumn('proportion', col('count')/test.count()).show()

*** TRAIN ***


+---+-----+--------------------+
|NPL|count|          proportion|
+---+-----+--------------------+
|  0| 7205|  0.9785413554257776|
|  1|  158|0.021458644574222464|
+---+-----+--------------------+

*** TEST ***


+---+-----+--------------------+
|NPL|count|          proportion|
+---+-----+--------------------+
|  0| 2308|  0.9800424628450106|
|  1|   47|0.019957537154989383|
+---+-----+--------------------+



In [19]:
# Saving the datasets in distinct .parquet files.
train.write.parquet('/kaggle/working/train.parquet', mode='overwrite')
test.write.parquet('/kaggle/working/test.parquet', mode='overwrite')

<h3 style='font-size:30px;font-style:italic'> Defining the Models' Metric</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Providing False Negatives would clearly be more harmful for the bank's equity than False Positives.         
        </li>
        <li> 
            Following conversations with the credit analysts, we've ended up defining the case's official metric
            as an f-score with $\beta=4$. So, we are giving to Recall an importance 4x higher than the Precision's.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Exploratory Data Analysis</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            With the data properly segregated, let's briefly analyze its content and see whether we can spot differences 
            among the classes. 
        </li>
    </ul>
</div>

In [20]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [21]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from IPython.core.display import HTML

# Creating the project's SparkSession.
spark = SparkSession.builder.appName('NPL').getOrCreate()

# Also, modifying the session's log level.
log_level = spark.sparkContext.setLogLevel('ERROR')

# This tiny config enables us to scroll along the DataFrame's columns.
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [22]:
train = spark.read.parquet('/kaggle/input/npl-train/train.parquet/')
train.show(5)

+---------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---+
|ID_CLIENT|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|NPL|
+---------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+---+
|      691|          F|           N|              N|           0|         67500.0|         Working|Secondary / secon...|           Married|House / apartment|    -20075|        -7013|         1|   

<h3 style='font-size:30px;font-style:italic'> Assessing the Classes' Incomes</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Let's analyze whether there is any revenue difference between individuals who produced an NPL and those who didn't. 
        </li>
        <li> 
            Since we are dealing with an continuous outcome from two independent samples, I'll use the z-score formula below to examine the 
            income differences:
            <center style='margin-top:20px'> 
                    $z=\frac{\overline{X}_{1}-\overline{X}_{2}}{S_{p}\sqrt{\frac{1}{n_{1}}+\frac{1}{n_{2}}}}$ | $S_{p}=\sqrt{\frac{(n_{1}-1)s_{1}^{2}+(n_{2}-1)s_{2}^{2}}{n_{1}+n_{2}-2}}$
           </center>
        </li>
    </ul>
</div>

In [23]:
# The formula above is only valid for samples with similar std's (0.5<s1/s2<2). So we ought to firstly guarantee that the incomes' std's
# are in accordance to that rule.
df_income_std = train.groupBy('NPL').agg({'AMT_INCOME_TOTAL':'std'})
df_income_std.show()

+---+------------------------+
|NPL|stddev(AMT_INCOME_TOTAL)|
+---+------------------------+
|  0|        99647.7510722535|
|  1|       96639.80531139452|
+---+------------------------+



In [24]:
# Because the std ratio is in the desired interval, we can proceed in using the formula.
list_std = [row['stddev(AMT_INCOME_TOTAL)'] for row in df_income_std.collect()]
list_std[0] / list_std[1]

1.0311253292695148

In [25]:
# By setting our alpha=.05, we can see that there is no statistical evidence that people with no NPL receive higher incomes
# than the other group.
from statsmodels.stats.weightstats import ztest

zeros = [row['AMT_INCOME_TOTAL'] for row in train.select('AMT_INCOME_TOTAL').where('NPL==0').collect()]
ones = [row['AMT_INCOME_TOTAL'] for row in train.select('AMT_INCOME_TOTAL').where('NPL==1').collect()]

ztest(zeros, ones, alternative='larger')[1] # Computing our p-value.

0.7972574858186637

<h3 style='font-size:30px;font-style:italic'> Valuable Assets Analysis</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The dataset contains columns representing whether the client possesses real state or a vehicle.
        </li>
        <li> 
            We can verify if people that own such properties have lower chance of producing an NPL, because they could sell them if they don't have enough 
            cash to pay the loans. 
        </li>
    </ul>
</div>

In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def own_valuable(col_car:str, col_realty:str)->str:
    '''
        Signs whether a client possesses a valuable asset that can be sold for paying their loans.
        
        Parameters
        ----------
        `col_car` str
            The value of the flag that indicates the ownership of a car.
        `col_realty`: str
            The value of the flag that indicates the ownership of a real state.
            
        Returns
        -------
        A flag indicating the possession of a car or real state (valuable assets). 
    '''
    if (col_car=='Y') or (col_realty=='Y'):
        return '1'
    return '0'

# Creating the DataFrame that will be used for conducting the Hypothesis Test.
df_valuable = train.select(own_valuable('FLAG_OWN_CAR', 'FLAG_OWN_REALTY').alias('OWN_VALUABLE'), 'NPL')
df_valuable.show(5)

+------------+---+
|OWN_VALUABLE|NPL|
+------------+---+
|           0|  0|
|           0|  0|
|           0|  0|
|           0|  0|
|           0|  0|
+------------+---+
only showing top 5 rows



In [27]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as ps_sum

window = Window.partitionBy('NPL')
gb_valuable = (df_valuable
     .groupBy(['NPL', 'OWN_VALUABLE'])
     .count()
     .withColumn('TOTAL_INSTANCES_NPL', ps_sum(col('count')).over(window))
     .withColumn('PROP_OWN_VALUABLE', col('count')/ps_sum(col('count'))
     .over(window)))

(gb_valuable
     .withColumnRenamed('count', 'INSTANCES_WITH_VALUABLE')
     .where('`OWN_VALUABLE`==1')
     .drop('OWN_VALUABLE')).show()

+---+-----------------------+-------------------+------------------+
|NPL|INSTANCES_WITH_VALUABLE|TOTAL_INSTANCES_NPL| PROP_OWN_VALUABLE|
+---+-----------------------+-------------------+------------------+
|  0|                   5747|               7201|0.7980835995000695|
|  1|                    126|                172|0.7325581395348837|
+---+-----------------------+-------------------+------------------+



In [28]:
from statsmodels.stats.proportion import test_proportions_2indep
df_valuable.where('`OWN_VALUABLE`==1').groupBy(['NPL']).count().show()

+---+-----+
|NPL|count|
+---+-----+
|  0| 5747|
|  1|  126|
+---+-----+



In [29]:
train.where('`DAYS_EMPLOYED`>=0').count()

1300

In [30]:
train.where('`OCCUPATION_TYPE` IS NULL AND `DAYS_EMPLOYED`>=0').count()

1300

In [31]:
train.where('`OCCUPATION_TYPE` IS NULL AND `DAYS_EMPLOYED`<0').count()

978

<p style='color:red'> Documentar GP e fazer o Teste. Hip</p>